# Bulk Operations - FORALL and BULK COLLECT

This notebook demonstrates **bulk operations** in elastic-script for efficient batch processing.

## Features

- **FORALL** - Execute an action for each element in a collection
- **SAVE EXCEPTIONS** - Continue processing on errors, collect failures
- **BULK COLLECT** - Collect ES|QL results into an array

## 1. Basic FORALL Usage

In [ ]:
-- Simple FORALL with procedure call
CREATE PROCEDURE process_batch()
BEGIN
    DECLARE items ARRAY = [
        {'id': 1, 'name': 'Item A'},
        {'id': 2, 'name': 'Item B'},
        {'id': 3, 'name': 'Item C'}
    ];
    
    FORALL item IN items CALL log_item(item);
    
    PRINT 'Processed ' || ARRAY_LENGTH(items) || ' items';
END PROCEDURE;

CREATE PROCEDURE log_item(doc DOCUMENT)
BEGIN
    PRINT 'Processing: ' || doc['name'];
END PROCEDURE;

CALL process_batch();

## 2. FORALL with Built-in Functions

In [ ]:
-- FORALL with function call (e.g., INDEX_DOCUMENT)
CREATE PROCEDURE index_batch()
BEGIN
    DECLARE docs ARRAY = [
        {'message': 'Log entry 1', 'level': 'INFO'},
        {'message': 'Log entry 2', 'level': 'WARN'},
        {'message': 'Log entry 3', 'level': 'ERROR'}
    ];
    
    FORALL doc IN docs INDEX_DOCUMENT('test-bulk-index', doc);
    
    PRINT 'Indexed ' || ARRAY_LENGTH(docs) || ' documents';
END PROCEDURE;

CALL index_batch();

## 3. FORALL with SAVE EXCEPTIONS

In [ ]:
-- Continue processing on errors with SAVE EXCEPTIONS
CREATE PROCEDURE process_with_errors()
BEGIN
    DECLARE items ARRAY = [
        {'id': 1, 'valid': true},
        {'id': 2, 'valid': false},
        {'id': 3, 'valid': true},
        {'id': 4, 'valid': false},
        {'id': 5, 'valid': true}
    ];
    
    FORALL item IN items CALL validate_item(item) SAVE EXCEPTIONS;
    
    PRINT 'Processing complete';
    PRINT 'Errors: ' || ARRAY_LENGTH(bulk_errors);
    
    IF ARRAY_LENGTH(bulk_errors) > 0 THEN
        DECLARE i NUMBER;
        FOR i IN 1..ARRAY_LENGTH(bulk_errors) LOOP
            PRINT '  Error at index ' || bulk_errors[i]['index'] || ': ' || bulk_errors[i]['error'];
        END LOOP;
    END IF;
END PROCEDURE;

CREATE PROCEDURE validate_item(item DOCUMENT)
BEGIN
    IF item['valid'] = false THEN
        THROW 'Validation failed for item ' || item['id'];
    END IF;
    PRINT 'Validated item ' || item['id'];
END PROCEDURE;

CALL process_with_errors();

## 4. BULK COLLECT

In [ ]:
-- Collect ES|QL results into an array
CREATE PROCEDURE collect_logs()
BEGIN
    BULK COLLECT INTO all_logs FROM logs-sample | LIMIT 10;
    
    PRINT 'Collected ' || ARRAY_LENGTH(all_logs) || ' logs';
    
    DECLARE i NUMBER;
    FOR i IN 1..ARRAY_LENGTH(all_logs) LOOP
        PRINT i || ': ' || all_logs[i]['level'] || ' - ' || SUBSTR(all_logs[i]['message'], 1, 50);
    END LOOP;
END PROCEDURE;

CALL collect_logs();

## 5. BULK COLLECT + FORALL Pipeline

In [ ]:
-- Collect, transform, and process
CREATE PROCEDURE etl_pipeline()
BEGIN
    BULK COLLECT INTO source_data FROM logs-sample | WHERE level = 'ERROR' | LIMIT 5;
    
    PRINT 'Collected ' || ARRAY_LENGTH(source_data) || ' error logs';
    
    FORALL log IN source_data CALL archive_error(log) SAVE EXCEPTIONS;
    
    PRINT 'Archived with ' || ARRAY_LENGTH(bulk_errors) || ' errors';
END PROCEDURE;

CREATE PROCEDURE archive_error(log DOCUMENT)
BEGIN
    PRINT 'Archiving: ' || SUBSTR(log['message'], 1, 40);
END PROCEDURE;

CALL etl_pipeline();

## 6. Combining with Cursors

In [ ]:
-- Process large datasets with cursors + FORALL
CREATE PROCEDURE batch_cursor_processing()
BEGIN
    DECLARE log_cursor CURSOR FOR FROM logs-sample | LIMIT 20;
    DECLARE batch ARRAY;
    DECLARE total_processed NUMBER = 0;
    DECLARE batch_num NUMBER = 0;
    
    OPEN log_cursor;
    FETCH log_cursor LIMIT 5 INTO batch;
    
    WHILE ARRAY_LENGTH(batch) > 0 LOOP
        SET batch_num = batch_num + 1;
        PRINT 'Processing batch ' || batch_num || ' (' || ARRAY_LENGTH(batch) || ' items)';
        
        FORALL item IN batch CALL process_log_entry(item) SAVE EXCEPTIONS;
        
        SET total_processed = total_processed + ARRAY_LENGTH(batch);
        FETCH log_cursor LIMIT 5 INTO batch;
    END LOOP;
    
    CLOSE log_cursor;
    PRINT 'Total processed: ' || total_processed;
END PROCEDURE;

CREATE PROCEDURE process_log_entry(entry DOCUMENT)
BEGIN
    PRINT '  -> ' || entry['level'];
END PROCEDURE;

CALL batch_cursor_processing();

## Reference

### FORALL Syntax

```sql
FORALL element IN collection action [SAVE EXCEPTIONS];
```

- `element`: Loop variable bound to each item
- `collection`: An ARRAY to iterate over
- `action`: CALL procedure(...) or function_call(...)
- `SAVE EXCEPTIONS`: Optional - continue on errors, collect in `bulk_errors`

### BULK COLLECT Syntax

```sql
BULK COLLECT INTO variable FROM esql_query;
```

- `variable`: Target ARRAY variable
- `esql_query`: ES|QL query to execute

### bulk_errors Structure

When using `SAVE EXCEPTIONS`, the `bulk_errors` variable contains:

| Field | Type | Description |
|-------|------|-------------|
| `index` | NUMBER | Position in collection where error occurred |
| `element` | ANY | The element that caused the error |
| `error` | STRING | Error message |
| `type` | STRING | Exception type |